# Como usar

## 1. Na aba 'Gerar o Sistema':escolha os parâmetros e clique em 'Gerar/Resolver' para gerar a equação matricial e resolvê-la.

## 2. Na aba 'Funções de Onda' escolha os parâmetros do plot e clique em 'Plotar'.

***

In [1]:
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from numba import jit
import ipywidgets as wid
from ipywidgets import interactive
from ipywidgets.embed import embed_minimal_html

RY=13.6056980659 #x*RY: Rydberg -> ev
ang=1.88973 #X/ang: raio de bohr -> angstroms

def diag(xx,N):
    global x,psi,prob,eig,V,L
    def vetores(xx,N):
        xmin,xmax=xx
        x,dx=np.linspace(xmin*ang,xmax*ang, N, retstep=True) #Define o vetor posição (raio de bohr)
        return x,dx,xmin,xmax
    
    @jit
    def diagH(H,V):
        H=H+V
        eig, psi=np.linalg.eigh(H) #Resolve o sistema e atribui-se: eig=autovalores, psi=autovetores
        prob=np.abs(psi)**2 #Atribui a densidade de probabilidade |Psi|² a variável psi, por conveniência
        return H,V,psi,prob,eig
    
    x,dx,xmin,xmax=vetores(xx,N)
    a=- 1/(dx**2) #Define 'a' por conveniência
    N=x.size #Armazena o tamanho do vetor posição e o comprimento do espaço que ele representa
    V=sparse.diags([0],[0],shape=(N,N)).toarray() #Cria uma matriz NxN com o potencial V(x)
    H=sparse.diags([a,-2*a,a],[-1,0,1],shape=(N,N)).toarray() #Cria a matriz Hamiltoniano
    H,V,psi,prob,eig=diagH(H,V)
    
    eig=eig*RY #Ryd->eV
    x=x/ang #RBohr->angstroms
    L=x.max()-x.min()
    
    print('\n #########Propriedades espaciais do sistema resolvido (angstroms)######### ')
    print(f"\n Largura do domínio =  {L:.2f}")
    print(f"\n dx =  {dx:.2f}")
    print("\n #########################################################################")
    print("\n Siga para a aba 'Funções de onda' para plotar o resultado.")
    
def plotall(NES,limx,wav):
    global V
    #V=np.diag(V,k=0)
    fig, ax= plt.subplots(figsize(9,16))
    nmin,nmax=NES
    ax.set_xlim([x.min()-limx, x.max()+ limx])
    ax.set_xlabel('$x (\AA$)')
    ax.set_ylabel('Auto energias (eV)')
    
    if wav==1:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x,prob[:,i]+eig[i], 
                         label=f'$E_{i}={eig[i]:2f}$ eV')
    elif wav==2:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x,np.real(psi[:,i])+eig[i], 
                         label=f'$E_{i}={eig[i]:2f}$ eV')
    elif wav==3:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x,np.imag(psi[:,i])+eig[i], 
                         label=f'$E_{i}={eig[i]:2f}$ eV')
            
    fig=plt.axvline(x=x.min(),color='k',ls='--')
    fig=plt.axvline(x=x.max(),color='k',ls='--')
    #fig=plt.plot(x,V,'k',label='V($x$)')
    leg=plt.legend(loc='upper right')
    plt.show()


lslider=wid.Layout(width='90%', display='flex');
sty = {'description_width': 'initial'};

MAIN=interactive(diag, {'manual': True}, 
                    xx=wid.FloatRangeSlider(
                        value=[-50.0, 50.0],
                        min=-150.0,
                        max=150.0,
                        step=1,
                        continuous_update=False,
                        orientation='horizontal',
                        readout_format='1d',
                        layout=lslider,
                        description=('$x_{min}-x_{max}$'),
                        style=sty), 
                    N=wid.IntSlider(value=500,
                        min=200,
                        max=2000,
                        step=10,
                        continuous_update=False, 
                        layout=lslider,
                        description=('Número de pontos: '),
                        style=sty))


PLOT=interactive(plotall, {'manual': True}, 
                    NES=wid.IntRangeSlider(
                        value=[0, 3],
                        min=0,
                        max=10,
                        step=1,
                        description='Plotar N. Estados:',
                        continuous_update=False,
                        orientation='horizontal',
                        readout_format='.1d',
                        layout=lslider,
                        style=sty),
                    limx=wid.IntSlider(
                        value=20,
                        min=0,
                        max=80,
                        step=1,
                        description='Margens em $x$:   ',
                        continuous_update=False,
                        layout=lslider,
                        style=sty),
                    wav=wid.Dropdown(
                        options=[('densidades de probabilidades',1),
                                 ('funções de onda (parte real)',2),
                                 ('funções de onda (parte imaginária)',3)],
                        value=1,
                        description='Plotar as',
                        continuous_update=False))

MAIN.children[2].description=('Gerar/Resolver');
PLOT.children[3].description=('Plotar');

tab=wid.Tab(children=[MAIN,PLOT])
tab.set_title(0, 'Gerar o Sistema')
tab.set_title(1, 'Funções de Onda')

display(tab)
